In [1]:
# 準備
from tensorflow.keras.layers import Dense , Flatten, Convolution2D , MaxPooling2D    # Dense 建立網路 , Convolution2D 建立卷基層 , MaxPooling2D 建立池化層
from tensorflow.keras.models import Sequential                                       # Sequential 初始化神經網路
from tensorflow.keras.preprocessing.image import ImageDataGenerator                  # 圖片生成器, 擴大訓練集

In [4]:
# 初始化 CNN
classifier = Sequential()  # 初始化神經網路

# 構建 卷積層, 定義 filter(特徵探測器)個數: 32個, filter大小: 3*3
# 建立第一層後要先寫清楚輸入的數據長什麼樣, 給他 64 * 64 初始化成統一格式 (圖片的特徵縮放)
classifier.add(Convolution2D(32, (3, 3), activation = 'relu', input_shape = (64, 64, 3)))   

# 對卷積後的特徵圖 構建 最大池化層, 縮減特徵圖的大小
classifier.add(MaxPooling2D(pool_size = (2, 2))) 

# 建立第二個捲積層與最大池化層
classifier.add(Convolution2D(32, (3, 3), activation = 'relu'))  
classifier.add(MaxPooling2D(pool_size = (2, 2))) 

# 將池化後的特徵圖拉平, 成為全連接層的輸入層
classifier.add(Flatten())

# 添加全連接層 (ANN的隱藏層)
# 隱藏層 定義神經元個數為 128 (實驗出來的)
classifier.add(Dense(units = 128, activation = 'relu', kernel_initializer = 'uniform'))
classifier.add(Dense(units = 128, activation = 'relu', kernel_initializer = 'uniform'))

# 添加輸出層
classifier.add(Dense(units = 1, activation = 'sigmoid'))

In [5]:
# 要先編譯神經網路 (定義損失函數，建立隨機梯度下降，學習率 ) 
classifier.compile(optimizer = 'adam' , loss = 'binary_crossentropy', metrics=['accuracy'])

In [6]:
# 對輸入的影像前處理

# 建立訓練集的生成器, 用來特徵縮放, 大小縮放, 水平翻轉
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
# 建立測試集生成器
test_datagen = ImageDataGenerator(rescale = 1./255)


# 將訓練集生成器用在訓練集上
training_set = train_datagen.flow_from_directory('data/training_set',
                                                 target_size = (64, 64),   # 圖片輸入進卷積層的大小
                                                 batch_size = 32,          # 一批一批生成的數目
                                                 class_mode = 'binary')    # 將圖片分成幾個類 (categorical: 多類)
# 將測試集生成器用在測試集上
test_set = test_datagen.flow_from_directory('data/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')
# 模型訓練
classifier.fit_generator(training_set,                 # 訓練集
                         steps_per_epoch = 250,        # 
                         epochs = 25,                
                         validation_data = test_set,   # 測試集
                         validation_steps = 62.5)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/25
250/250 [==============================] - 35s 141ms/step - loss: 0.6764 - accuracy: 0.5717 - val_loss: 0.6117 - val_accuracy: 0.6695
Epoch 2/25
250/250 [==============================] - 34s 137ms/step - loss: 0.6009 - accuracy: 0.6805 - val_loss: 0.5958 - val_accuracy: 0.6725
Epoch 3/25
250/250 [==============================] - 37s 150ms/step - loss: 0.5638 - accuracy: 0.7141 - val_loss: 0.5668 - val_accuracy: 0.7125
Epoch 4/25
250/250 [==============================] - 32s 127ms/step - loss: 0.5364 - accuracy: 0.7314 - val_loss: 0.5215 - val_accuracy: 0.7475
Epoch 5/25
250/250 [==============================] - 30s 119ms/step - loss: 0.5080 - accuracy: 0.7498 - val_loss: 0.4935 - val_accuracy: 0.7615- loss:
Epoch 6/25
250/250 [==============================] - 31s 125ms/step - loss: 0.4762 - accuracy: 0.7734 - val

In [18]:
import cv2
import numpy as np
img = cv2.imread('dog.jpg')

# predict
pred = classifier.predict(img)
class_number = np.argmax(pred, axis=1)

class_number

ValueError: in user code:

    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1462 predict_function  *
        return step_function(self, iterator)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1445 run_step 